Since all the data before 2019 is behind a login screen I need to code a way to logon to the website to grab the data. 

The very first thing to do is import the python packages BeautifulSoup, requests, pandas, and creds.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import creds

Gathered the url and other information by using the inspect network element on google.

In [2]:
cookies = {
    'MCPopupClosed': 'yes',
    '_gid': 'GA1.2.1783712766.1660157331',
    'PHPSESSID': '8lv90mcln6odafu4ct8v488tvloe3olj_premium',
    '_gat': '1',
    '_ga_3ZZTGEXZ3B': 'GS1.1.1660168631.8.1.1660175234.0',
    '_ga': 'GA1.2.1968697907.1659831804',
    '_gali': 'contactForm',
}

headers = {
    'authority': 'www.spotrac.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'MCPopupClosed=yes; _gid=GA1.2.1783712766.1660157331; PHPSESSID=8lv90mcln6odafu4ct8v488tvloe3olj_premium; _gat=1; _ga_3ZZTGEXZ3B=GS1.1.1660168631.8.1.1660175234.0; _ga=GA1.2.1968697907.1659831804; _gali=contactForm',
    'origin': 'https://www.spotrac.com',
    'referer': 'https://www.spotrac.com/signin/',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

data = {
    'redirect': 'nfl/rankings/2015/cash/',
    'email': creds.email,
    'password': creds.password,
}
data2 = {
    'ajax' : 'true',
    'mobile': 'false'
}

Using the requests package and the data collected from earlier I log into the website and navigate to the page with the data I want to collect.

In [3]:
url = ('https://www.spotrac.com/nfl/rankings/2015/cash')
with requests.session() as s:
    s.post('https://www.spotrac.com/signin/submit/', data=data)
    r = s.post(url, data=data2)
    soup = BeautifulSoup(r.content, 'html.parser')

Created a list to attach the data to.

In [4]:
rows_list = list()

With BeautifulSoup I gather all the data from the table I wanted.

In [5]:
nfl_table = soup.find('table', class_ = 'datatable captracker noborder')
for body in nfl_table.find_all('tbody'):
    rows = body.find_all('tr')
    for row in rows:
         pl_name = row.find('a', class_ ='team-name').text.strip()
         pl_team_name = row.find('div', class_ = 'rank-position').text.strip()
         pl_position = row.find('td', class_ = 'center med').text.strip()
         pl_salary = row.find_all('span', class_ = 'info')
         for span in pl_salary:
            span = span.text.replace('usd','').strip()      
         rows_list.append([pl_name,pl_team_name,pl_position,span])

Used the list from earlier and transformed it into a dataframe with column names using pandas and then converted the dataframe into a csv file.

In [7]:
df_nfl=pd.DataFrame(rows_list,columns=['Name','Team','POS', 'Salary'])
df_nfl.to_csv('NFL 2015 Salary.csv', index=False)
nfl_file=pd.read_csv('NFL 2015 Salary.csv')
df_nfl.head()


,Name,Team,POS,Salary
0,Eli Manning,NYG,QB,"$37,010,004"
1,Ben Roethlisberger,PIT,QB,"$35,257,482"
2,Philip Rivers,SD,QB,"$32,007,656"
3,Russell Wilson,SEA,QB,"$31,722,469"
4,A.J. Green,CIN,WR,"$26,750,000"
